In [ ]:
import math

def simplify(coords, first, last, sqTolerance) :
    maxSqDist = sqTolerance
    mid = (last - first) >> 1
    minPosToMid = last - first
    index

    ax = coords[first]
    ay = coords[first + 1]
    bx = coords[last]
    by = coords[last + 1]
    for i in range(first + 3, last, 3):
        d = getSqSegDist(coords[i], coords[i + 1], ax, ay, bx, by)

        if (d > maxSqDist) :
            index = i
            maxSqDist = d

        elif (d == maxSqDist) :
            posToMid = math.abs(i - mid)
            if (posToMid < minPosToMid) :
                index = i
                minPosToMid = posToMid
            
    if (maxSqDist > sqTolerance) :
        if (index - first > 3):
            simplify(coords, first, index, sqTolerance)
        coords[index + 2] = maxSqDist
        if (last - index > 3):
            simplify(coords, index, last, sqTolerance)
    

def getSqSegDist(px, py, x, y, bx, by) :
    dx = bx - x
    dy = by - y
    if (dx != 0 or dy != 0) :
        t = ((px - x) * dx + (py - y) * dy) / (dx * dx + dy * dy)
        if (t > 1) :
            x = bx
            y = by
        elif (t > 0) :
            x += dx * t
            y += dy * t
    dx = px - x
    dy = py - y
    return dx * dx + dy * dy

In [ ]:
# def createFeature(id, type, geom, tags) {
#     const feature = {
#         id: id == null ? null : id,
#         type,
#         geometry: geom,
#         tags,
#         minX: Infinity,
#         minY: Infinity,
#         maxX: -Infinity,
#         maxY: -Infinity
#     };

#     if (type === 'Point' || type === 'MultiPoint' || type === 'LineString') {
#         calcLineBBox(feature, geom);

#     } else if (type === 'Polygon') {
#         // the outer ring (ie [0]) contains all inner rings
#         calcLineBBox(feature, geom[0]);

#     } else if (type === 'MultiLineString') {
#         for (const line of geom) {
#             calcLineBBox(feature, line);
#         }

#     } else if (type === 'MultiPolygon') {
#         for (const polygon of geom) {
#             // the outer ring (ie [0]) contains all inner rings
#             calcLineBBox(feature, polygon[0]);
#         }
#     }

#     return feature;
# }

# function calcLineBBox(feature, geom) {
#     for (let i = 0; i < geom.length; i += 3) {
#         feature.minX = Math.min(feature.minX, geom[i]);
#         feature.minY = Math.min(feature.minY, geom[i + 1]);
#         feature.maxX = Math.max(feature.maxX, geom[i]);
#         feature.maxY = Math.max(feature.maxY, geom[i + 1]);
#     }
# }


In [ ]:
def projectX(x):
    return x / 360 + 0.5

def projectY(y):
    sin = math.sin(y * math.pi / 180)
    y2 = 0.5 - 0.25 * math.log((1 + sin) / (1 - sin)) / math.pi
    if y2 < 0:
        return 0
    else:
        if y2 > 1:
            return 1
        else:
            return y2

In [ ]:
def convertLine(ring, out, tolerance, isPolygon):
    x0, y0 = 0,0
    size = 0

    for j in range(ring.length):
        x = projectX(ring[j][0])
        y = projectY(ring[j][1])
        out.extend([x,y,0])

        if (j > 0):
            if (isPolygon):
                size += (x0 * y - x * y0) / 2
        else:
                size += math.sqrt(pow(x - x0, 2) + pow(y - y0, 2))
        x0 = x
        y0 = y

    last = len(out) - 3
    out[2] = 1
    simplify(out, 0, last, tolerance)
    out[last + 2] = 1

    out.size = math.abs(size)
    out.start = 0
    out.end = out.size
    
def convertLines(rings, out, tolerance, isPolygon) :
    for ring in rings:
        geom = []
        convertLine(ring, geom, tolerance, isPolygon)
        out.append(geom)
    


def convertPoint(coords, out) :
    out.extend([projectX(coords[0]), projectY(coords[1]), 0])




In [ ]:
def convertFeature(features, geojson, options, index) :
    if geojson['geometry'] is None:
        return None

    coords = geojson['geometry']['coordinates']
    geotype = geojson['geometry']['type']
    tolerance = pow(options['tolerance'] / ((1 << options['maxZoom']) * options['extent']), 2)
    geometry = []
    id = geojson['id']
    if (options['promoteId']) :
        id = geojson['properties'][options['promoteId']]
    elif (options['generateId']) :
        if index is None:
            id = 0
        else:
            id = index
      
    if geotype == 'Point':
        convertPoint(coords, geometry)
    elif geotype == 'MultiPoint':
        for p in coords :
            convertPoint(p, geometry)
    elif geotype == 'LineString' :
        convertLine(coords, geometry, tolerance, False)
    elif geotype == 'MultiLineString' :
        if options['lineMetrics'] :
            for line in coords:
                geometry = []
                convertLine(line, geometry, tolerance, False)
                features.append(createFeature(id, 'LineString', geometry, geojson['properties']))
            return
        else:
            convertLines(coords, geometry, tolerance, False)
    elif geotype == 'Polygon':
           convertLines(coords, geometry, tolerance, True)
    elif geotype == 'MultiPolygon':
        for polygon in coords:
            newPolygon = []
            convertLines(polygon, newPolygon, tolerance, true)
            geometry.append(newPolygon)
    elif (geotype == 'GeometryCollection') :
        for singleGeometry in geojson['geometry']['geometries']:
            singleGeojson = {
               'id' : id,
               'geometry' : singleGeometry,
               'properties': geojson['properties']
            }
            convertFeature(features, singleGeojson, options, index)
        return
    else :
         raise ValueError('Incorrect Geometry type')
    features.append(createFeature(id, geotype, geometry, geojson['properties']))


In [ ]:
#   def convert(data, options) :
#      features = [];
#     if (data.type === 'FeatureCollection') :
#         for (let i = 0; i < data.features.length; i++) :
#             convertFeature(features, data.features[i], options, i);
        

#      else if (data.type === 'Feature') :
#         convertFeature(features, data, options);

#      else :
#         // single geometry or a geometry collection
#         convertFeature(features, :geometry: data, options);
    

#     return features;


In [2]:
# for i in range(0,6,2)
# print(i)

0
2
4
